<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import skimage
from skimage import io
import matplotlib.pyplot as plt
import json
import cv2
import numpy as np
import matplotlib
import os

In [8]:
DATA_DIR= "./data/"

def integrate_parallel(dataset = 'train'):

    mask = skimage.io.imread(DATA_DIR+'mask.png')
    if not os.path.isdir(DATA_DIR+'integrated_imgs_' + dataset):
        os.makedirs(DATA_DIR+'integrated_imgs_' + dataset)
    for folder in os.listdir(DATA_DIR + dataset):
        if folder.startswith(dataset):
            if not os.path.isdir(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder):
                os.makedirs(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder)
            with open(DATA_DIR + dataset + '/' + folder + '/homographies.json') as f:
                homographies = json.load(f)
            for time_point in [0,1,2,3,4,5,6]:
                warped_imgs = []
                for camera in ['B01', 'B02', 'B03', 'B04', 'B05', 'G01', 'G02', 'G03', 'G04', 'G05']:
                    img = skimage.io.imread(DATA_DIR + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '.png')
                    img = img[..., :3].astype(np.float32)/255
                    img[mask==0] = np.nan

                    homography = np.array(homographies[str(time_point) + '-' + camera])
                    warped_img = img.copy()
                    for dim in range(3):
                        warped_img[:,:,dim] = cv2.warpPerspective(img[:,:,dim],homography,img.shape[:2],borderMode = cv2.BORDER_CONSTANT,borderValue = np.nan) 
                    warped_imgs.append(warped_img)

                integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)

                integrated_img_nan = integrated_img.copy()
                for dim in range(3):
                    integrated_img_nan[:,:,dim] = np.where(np.isnan(warped_imgs).any(axis = 0).all(axis = 2), np.nan, integrated_img[:,:,dim])

            #     plt.figure()
            #     plt.imshow(integrated_img)
            #     plt.figure()
            #     plt.imshow(integrated_img_nan)
            
                matplotlib.image.imsave(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '_integrated.png', integrated_img)
                matplotlib.image.imsave(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '_integrated_nan.png', integrated_img_nan)


In [9]:
def integrate_all(dataset = 'train'):

    mask = skimage.io.imread(DATA_DIR + 'mask.png')
    if not os.path.isdir(DATA_DIR + 'integrated_imgs_' + dataset):
        os.makedirs(DATA_DIR + 'integrated_imgs_' + dataset)
    for folder in os.listdir(DATA_DIR + '' + dataset):
        if folder.startswith(dataset):
            if not os.path.isdir(DATA_DIR + 'integrated_imgs_' + dataset + '/' + folder):
                os.makedirs(DATA_DIR + 'integrated_imgs_' + dataset + '/' + folder)
            with open(DATA_DIR + '' + dataset + '/' + folder + 'homographies.json') as f:
                homographies = json.load(f)
            warped_imgs = []
            for time_point in [0,1,2,3,4,5,6]:
                for camera in ['B01', 'B02', 'B03', 'B04', 'B05', 'G01', 'G02', 'G03', 'G04', 'G05']:
                    img = skimage.io.imread(DATA_DIR + '' + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '.png')
                    img = img[..., :3].astype(np.float32)/255
                    img[mask==0] = np.nan

                    homography = np.array(homographies[str(time_point) + '-' + camera])
                    warped_img = img.copy()
                    for dim in range(3):
                        warped_img[:,:,dim] = cv2.warpPerspective(img[:,:,dim],homography,img.shape[:2],borderMode = cv2.BORDER_CONSTANT,borderValue = np.nan) 
                    warped_imgs.append(warped_img)

            integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)

            integrated_img_nan = integrated_img.copy()
            for dim in range(3):
                integrated_img_nan[:,:,dim] = np.where(np.isnan(warped_imgs).any(axis = 0).all(axis = 2), np.nan, integrated_img[:,:,dim])

    #         plt.figure()
    #         plt.imshow(integrated_img)
    #         plt.figure()
    #         plt.imshow(integrated_img_nan)
    
            matplotlib.image.imsave(DATA_DIR + 'integrated_imgs_' + dataset + '/' + folder + '/all_integrated.png', integrated_img)
            matplotlib.image.imsave(DATA_DIR + 'integrated_imgs_' + dataset + '/' + folder + '/all_integrated_nan.png', integrated_img_nan)


In [10]:
integrate_parallel('train')
integrate_parallel('test')
integrate_parallel('validation')

integrate_all('train')
integrate_all('test')
integrate_all('validation')

/var/folders/_c/75lstqlj3n5cllxr3qx75tc00000gn/T/ipykernel_33290/3107136023.py:27: RuntimeWarning: Mean of empty slice
  integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)


FileNotFoundError: [Errno 2] No such file or directory: './data/train/train-1-5homographies.json'